## Erstellung der SQL Datei

In [1]:
import pandas as pd

# Zielspalten
usecols = ['tconst', 'titleType', 'primaryTitle', 'startYear', 'genres']

# Basics-Datei in kleinen Portionen einlesen
chunks = pd.read_csv(
    "../data/raw/title.basics-2.tsv",
    sep='\t',
    dtype=str,
    na_values='\\N',
    usecols=usecols,
    chunksize=100_000
)

# Nur relevante Zeilen aus jedem Chunk sammeln
filtered_chunks = []
for chunk in chunks:
    movies = chunk[chunk['titleType'] == 'movie']
    movies = movies[movies['startYear'].notna()]
    filtered_chunks.append(movies)

# Alle Ergebnisse zusammenfügen
basics = pd.concat(filtered_chunks, ignore_index=True)


In [2]:
ratings = pd.read_csv(
    "../data/raw/title.ratings.tsv",
    sep='\t',
    dtype=str,
    na_values='\\N'
)

ratings = ratings[['tconst', 'averageRating', 'numVotes']]
ratings['averageRating'] = pd.to_numeric(ratings['averageRating'], errors='coerce')
ratings['numVotes'] = pd.to_numeric(ratings['numVotes'], errors='coerce')

In [3]:
import sqlite3

conn = sqlite3.connect("../data/raw/imdb.sqlite")  # Pfad anpassen, falls nötig
basics.to_sql("basics", conn, if_exists="replace", index=False)
ratings.to_sql("ratings", conn, if_exists="replace", index=False)
conn.close()